In [1]:
import numpy as np
import datasets
from scipy.optimize import minimize
from scipy.special import expit
from softmax import softmax, log_softmax
import theano
from theano import tensor as T
import tensorflow as tf
import sklearn.preprocessing

In [2]:
X, y = datasets.htwt()
Y = datasets.one_hot(y)
N, D = X.shape
N, C = Y.shape

In [9]:
w = T.dvector()
loss = T.sum(T.nnet.binary_crossentropy(T.nnet.sigmoid(T.dot(X, w)), y))
loss1 = theano.function([w], loss)

def loss2(w):
    mus = expit(X.dot(w))
    return -sum([y[i] * np.log(mu) + (1 - y[i]) * np.log(1 - mu) for i, mu in enumerate(mus)])

params = np.array([.3] * D)
loss1(params), loss2(params)

(array(8004.900000000001), nan)

In [6]:
w = T.dvector()
loss = T.sum(T.nnet.binary_crossentropy(T.nnet.sigmoid(T.dot(X, w)), y))
grad = theano.function([w], T.grad(loss, w))
hess = theano.function([w], T.hessian(loss, w))    
loss = theano.function([w], loss)

def xloss(w):
    mus = expit(X.dot(w))
    return -sum([y[i] * np.log(mu) + (1 - y[i]) * np.log(1 - mu) for i, mu in enumerate(mus)])

params = [2] * D
w = minimize(loss, params).x
print w
w = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess).x
print w
yhat = np.argmax(expit(X.dot(w)))
print '%0.3f' % np.mean(yhat != y)

[-0.18314134  0.08043465]
[ nan  nan]
0.348


In [59]:
sfit = sklearn.preprocessing.StandardScaler().fit(X)

decode = lambda params: T.concatenate((params, T.zeros(D))).reshape((D, C))    
z = T.dvector()
W = decode(z)
loss = T.sum(T.nnet.categorical_crossentropy(T.nnet.softmax(T.dot(sfit.transform(X), W)), Y))
grad = theano.function([z], T.grad(loss, z))
hess = theano.function([z], T.hessian(loss, z))    
loss = theano.function([z], loss)

params = [0] * D * (C - 1)
params = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess).x
params = minimize(loss, params).x

decode = lambda params: np.array(list(params) + [0] * D).reshape((D, C))
W = decode(params)
yhat = np.argmax(softmax(sfit.transform(X).dot(W)), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.186


In [61]:
def crossentropy_loss(X, Y, decode):
    N, D = X.shape
    N, C = Y.shape
    
    def loss(params):
        W, b = decode(params)
        return -np.trace(Y.dot(log_softmax(X.dot(W) + b).T))
#        return -sum([Y[i].dot(ll) for i, ll in enumerate(log_softmax(X.dot(W) + b))])

    def grad(params):
        W, b = decode(params)
        Z = softmax(X.dot(W) + b) - Y
        dW = sum([np.kron(z, X[i]) for i, z in enumerate(Z)])
        dW = np.array(dW.reshape((C, C))).T
        db = np.sum(Z, axis = 0)
        return np.array(list(dW.ravel()) + list(db.ravel()))
    
    def hess(params):
        return np.array([0] * 36).reshape((6, 6))
        W, b = decode(params)
        Z = softmax(X.dot(W) + b)
        o = lambda x: np.outer(x, x)
        dW = sum([np.kron(np.diag(z) - o(z), o(X[i])) for i, z in enumerate(Z)])
        db = sum([np.diag(z) - o(z) for i, z in enumerate(Z)])
        return np.array(list(dW.ravel()) + list(db.ravel()))
        
    return loss, grad, hess

params = [0] * (D + 1) * C
decode = lambda params: (params[:-C].reshape((D, C)), params[-C:])
loss, grad, hess = crossentropy_loss(X, Y, decode)

# params = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess).x
params = minimize(loss, params).x
W, b = decode(params)
yhat = np.argmax(softmax(X.dot(W) + b), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.119


In [62]:
W = tf.Variable(tf.zeros((D, C)))
b = tf.Variable(tf.zeros(C))

loss = tf.nn.softmax_cross_entropy_with_logits(tf.matmul(X, W) + b, Y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    sess.run(optimizer)
    W, b = W.eval(), b.eval()

yhat = np.argmax(softmax(X.dot(W) + b), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.348


In [67]:
W = theano.shared(np.zeros((D, C)))
b = theano.shared(np.zeros(C))
tx = T.dmatrix()
ty = T.dmatrix()

loss = T.sum(T.nnet.categorical_crossentropy(T.nnet.softmax(T.dot(tx, W) + b), ty))
learning_rate = 0.1
optimizer = theano.function([tx, ty], updates =
    [[p, p - T.grad(loss, p) * learning_rate] for p in [W, b]]
)

optimizer(X, Y)
W, b = W.get_value(), b.get_value()
yhat = np.argmax(softmax(X.dot(W) + b), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.348
